# Project 1 - OpenData Octopi

#### This Jupyter Notebook will show changes and code written by Nico. 

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import requests
import json